In [1]:
import sys,io,os

In [2]:
import pandas as pd
import numpy as np

In [3]:
wd = "/home/sakae/Windows/Python/FOSNLP/"
os.chdir(wd)

In [4]:
path_state = "./material/state_transition.tsv"
path_repr = "./material/repr_state_obesrvation.tsv"

In [5]:
df_state = pd.read_csv(path_state, index_col=0, header=0, delimiter="\t")
df_repr = pd.read_csv(path_repr, index_col=0, header=0, delimiter="\t")

In [6]:
df_repr

,AT,BEZ,IN,NN,VB,PERIOD
repr/state,,,,,,
bear,0,0,0,10,43,0
is,0,10065,0,0,0,0
move,0,0,0,36,133,0
on,0,0,5484,0,0,0
president,0,0,0,382,0,0
progress,0,0,0,108,4,0
the,69016,0,0,0,0,0
.,0,0,0,0,0,48809
<OOV>,51975,0,125050,133635,20796,458


In [7]:
df_state

,AT,BEZ,IN,NN,VB,PERIOD
from/to,,,,,,
AT,0,0,0,48636,0,19
BEZ,1973,0,426,187,0,38
IN,43322,0,1325,17314,0,185
NN,1067,3720,42470,11773,614,21392
VB,6072,42,4758,1476,129,1522
PERIOD,8016,75,4656,1329,954,0


## conversion from observation to probability
* `*_state`: state transition probability
* `*_repr`: observation probability

In [8]:
complement = True
if complement:
    s_total = df_repr.sum(axis=0)
    df_state["<OTHER>"] = s_total - df_state.sum(axis=1)

In [9]:
df_state_trans_prob = df_state.apply(lambda s: s / df_state.sum(axis=1))

In [10]:
if complement:
    df_state_trans_prob.drop(columns="<OTHER>", inplace=True)

In [11]:
lst_tup_trans_prob = []
for s_0 in df_state_trans_prob.index:
    for s_1 in df_state.index:
        tup_trans_prob = (s_0, s_1, df_state_trans_prob.loc[s_0, s_1])
        lst_tup_trans_prob.append(tup_trans_prob)

In [12]:
df_repr_prob = df_repr.apply(lambda s: s/df_repr.sum(axis=0), axis=1)

In [13]:
lst_tup_repr_prob = []
for s in df_repr_prob.columns:
    for r in df_repr_prob.index:
        tup_trans_prob = (s, r, df_repr_prob.loc[r, s])
        lst_tup_repr_prob.append(tup_trans_prob)

## instanciate HMM class

In [14]:
# from Tagger.HMM import PlainHMM
from Tagger.HMM import HMM

In [15]:
model = HMM()

* set state/representation symbols

In [16]:
model.set_repr(representations=df_repr_prob.index)
model.set_state(states=df_state_trans_prob.index, state_bos="PERIOD")

* set state/representation probabilities
    * state transition probability
    * representation probability

In [17]:
model.set_repr_obs_prob(lst_tuple_repr_prob=lst_tup_repr_prob)
model.set_state_trans_prob(lst_tuple_trans_prob=lst_tup_trans_prob)

/media/sakae/D/Linux/Python/FOSNLP/Tagger/HMM.py:109: RuntimeWarning: divide by zero encountered in log
  self._mat_log_repr_obs_prob = np.log(self._mat_repr_obs_prob)
/media/sakae/D/Linux/Python/FOSNLP/Tagger/HMM.py:97: RuntimeWarning: divide by zero encountered in log
  self._mat_log_state_trans_prob = np.log(self._mat_state_trans_prob)


## Evaluate negative log likelihood of specified sequence of tags/words

In [35]:
lst_repr = "The bear is on the move .".lower().split()
lst_state_1 = "AT NN BEZ IN AT NN PERIOD".split()
lst_state_2 = "AT NN BEZ IN AT VB PERIOD".split()

In [36]:
s = " ".join([r + "/" + s for r, s in zip(lst_repr, lst_state_1)])
print("case 1: %s" % s)
nll_1 = model.evaluate(lst_state_1, lst_repr, verbose=True)
print("probability: %e" % np.exp(-nll_1))

s = " ".join([r + "/" + s for r, s in zip(lst_repr, lst_state_2)])
print("\ncase 2: %s" % s)
nll_2 = model.evaluate(lst_state_2, lst_repr, verbose=True)
print("probability: %e" % np.exp(-nll_2))

case 1: the/AT bear/NN is/BEZ on/IN the/AT move/NN ./PERIOD
p(s_t+1|s_t): 0.162705, p(r_t|s_t): 0.570423
p(s_t+1|s_t): 0.401980, p(r_t|s_t): 0.000075
p(s_t+1|s_t): 0.027726, p(r_t|s_t): 1.000000
p(s_t+1|s_t): 0.042325, p(r_t|s_t): 0.042012
p(s_t+1|s_t): 0.331883, p(r_t|s_t): 0.570423
p(s_t+1|s_t): 0.401980, p(r_t|s_t): 0.000268
p(s_t+1|s_t): 0.159438, p(r_t|s_t): 0.990704
probability: 4.421454e-16

case 2: the/AT bear/NN is/BEZ on/IN the/AT move/VB ./PERIOD
p(s_t+1|s_t): 0.162705, p(r_t|s_t): 0.570423
p(s_t+1|s_t): 0.401980, p(r_t|s_t): 0.000075
p(s_t+1|s_t): 0.027726, p(r_t|s_t): 1.000000
p(s_t+1|s_t): 0.042325, p(r_t|s_t): 0.042012
p(s_t+1|s_t): 0.331883, p(r_t|s_t): 0.570423
p(s_t+1|s_t): 0.000000, p(r_t|s_t): 0.006341
p(s_t+1|s_t): 0.072559, p(r_t|s_t): 0.990704
probability: 0.000000e+00


## Predict most probable sequence of tags of specified words

In [21]:
lst_repr = "The bear is on the move .".lower().split()

In [29]:
nll, lst_state_map = model.predict(representations=lst_repr)

/usr/local/pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [33]:
s = " ".join([r + "/" + s for r, s in zip(lst_repr, lst_state_map)])
print("predicted sequence of tags: %s" % s)
print("probability: %e" % np.exp(-nll))

predicted sequence of tags: the/AT bear/NN is/BEZ on/IN the/AT move/NN ./PERIOD
probability: 4.421454e-16
